# Models and likelihood 

### George Tzanetakis, University of Victoria 

In this notebook we will introduce the idea of a model and how we can calculate the likelihood of a model given some data. 

## A random variable class 

Define a helper random variable class based on the scipy discrete random variable functionality providing both numeric and symbolic RVs. You don't need to look at the implementation - the usage will be obvious through the examples below. 

In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np 

class Random_Variable: 
    
    def __init__(self, name, values, probability_distribution): 
        self.name = name 
        self.values = values 
        self.probability_distribution = probability_distribution 
        if all(type(item) is np.int_ for item in self.values): 
            self.type = 'numeric'
            self.rv = stats.rv_discrete(name = name, 
                        values = (values, probability_distribution))
        elif all(type(item) is str for item in values): 
            self.type = 'symbolic'
            self.rv = stats.rv_discrete(name = name, 
                        values = (np.arange(len(values)), probability_distribution))
            self.symbolic_values = values 
        else: 
            self.type = 'undefined'
            
    def sample(self,size): 
        if (self.type =='numeric'): 
            return self.rv.rvs(size=size)
        elif (self.type == 'symbolic'): 
            numeric_samples = self.rv.rvs(size=size)
            mapped_samples = [self.values[x] for x in numeric_samples]
            return mapped_samples 
        
    def prob_of_value(self, value): 
        indices = np.where(self.values == value)
        return self.probability_distribution[indices[0][0]]

            

Lets start by creating a random variable corresponding to a 6-faced dice where there are two faces with the numbers 1,2 and 3 therefore each number appears with equal probability. We can generate random samples from this model. 

In [12]:
values = np.int64([1, 2, 3])
probabilities = [2/6., 2/6., 2/6.]
dice1 = Random_Variable('dice1', values, probabilities)
samples = dice1.sample(30)
print(samples)

[2 3 3 2 1 1 1 1 3 2 2 2 1 2 2 2 3 1 3 2 1 1 3 3 2 1 2 3 1 1]


Lets also create a random variable where three of the faces have the number 2, 2 faces have the number 1 and 1 face has the number 3. We can also generate random samples from this model. 

In [13]:
values = np.int_([1, 2, 3])
probabilities = [2./6, 3./6, 1./6]
dice2 = Random_Variable('dice2', values, probabilities)
samples = dice2.sample(30)
print(samples)

[3 2 2 1 1 2 2 2 3 2 3 1 2 2 2 1 2 2 2 2 2 2 2 2 3 2 1 2 1 1]


The likelihood of a sequence of samples given a model can be obtained by taking the product of the corresponding 
probabilities. We can see that for this particular sequence of data the likelihood of the model for dice2 is higher. So if we have some data and some specific models we can select the model with the highest likelihood. 

In [9]:
data = [1,2,2,1,1,3,1,2,3,2]
print(dice1.prob_of_value(1))
print(dice1.prob_of_value(3))
print(dice2.prob_of_value(3))

def likelihood(data, model):
    likelihood = 1.0 
    for d in data: 
        likelihood *= model.prob_of_value(d)
    return likelihood 
    
print("Likelihood for dice1: %f" % likelihood(data,dice1))
print("Likelihood for dice2: %f" % likelihood(data,dice2))


0.3333333333333333
0.3333333333333333
0.16666666666666666
Likelihood for dice1: 0.000017
Likelihood for dice2: 0.000021


In the case above we examined two possible models. One could ask the question of all possible models for a particular problem can we find the one with the highest likelihood ? If we have a dice with six faces that can only have the number 1, 2, and 3 then there is a finite amount of models and we can calculate their likelihoods as we did above. However if we relax the requirement to have a dice and simply have the values 1,2 and 3 but with arbitrary associated probabilities then we have an infinte number of possible models. Without going into the math it turns out that at least for this particular case the model that will have the maximum likelihood can be simply obtained by counting the relative frequencies of the values in the data. This is called maximum likelihood estimation of model parameters and is something we will revisit later. 

In [14]:
import collections 

data = [1,2,2,1,1,3,1,2,3,2]
counts = collections.Counter(data)
print(counts)
est_probability_distribution = [counts[1]/float(len(data)), counts[2]/float(len(data)), counts[3]/float(len(data))]
print(est_probability_distribution)

Counter({1: 4, 2: 4, 3: 2})
[0.4, 0.4, 0.2]


We can use this approach to create a new model from this data and generate new samples. 
When using probabilistic models in machine learning, we use data to estimate the parameters 
of typically much more complex models (training) and then use those models to estimate probabilities and likelihoods for other sets of data (testing). 


In [15]:
values = np.int_([1, 2, 3])
probabilities = est_probability_distribution 
model = Random_Variable('model', values, probabilities)
samples = model.sample(30)
print(samples)

[1 1 1 2 1 1 1 1 1 2 1 3 3 1 1 1 1 2 2 1 1 1 2 2 1 3 2 1 1 1]
